In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import utils 
utils.init_logging()

import polars as pl
from polars import col, lit
import altair as alt
import simrs
import input_builder as b
import scenarios
import cli
import analysis

In [3]:
# scenario_input = scenarios.barbell_scenario(ticks=100, num_agents=10)
# (actions, agents, markets, events) = scenarios.run_scenario(scenario)

scenario = scenarios.load_scneario()
(actions, agents, markets, events) = (scenario.actions, scenario.agents, scenario.markets, scenario.events)

In [4]:
print("agents") 
agents.head()

agents


behavior,cargo,coins,id,pos,tick
str,str,f64,str,str,i64
"""Exhaustive""",null,1000.0,"""a9""","""la""",0
"""Exhaustive""",null,1000.0,"""a6""","""la""",0
"""Exhaustive""",null,1000.0,"""a2""","""la""",0
"""Exhaustive""",null,1000.0,"""a7""","""la""",0
"""Exhaustive""",null,1000.0,"""a5""","""la""",0


In [5]:
print("markets") 
markets.head()

markets


consumption,good,port,price,production,supply,tick
f64,str,str,f64,f64,f64,i64
100.0,"""Wheat""","""m3""",100.0,100.0,1000.0,0
100.0,"""Wheat""","""ra""",98.039216,100.0,1020.0,0
100.0,"""Wheat""","""la""",102.040816,100.0,980.0,0
100.0,"""Wheat""","""m1""",102.040816,100.0,980.0,0
100.0,"""Wheat""","""m2""",100.0,100.0,1000.0,0


In [6]:
print("actions")
actions.head(10)

actions


action,agent_id,port_id,tick,good
str,str,str,i64,str
"""Move""","""a9""","""m1""",0,null
"""Move""","""a6""","""m1""",0,null
"""Move""","""a2""","""m1""",0,null
"""Move""","""a7""","""m1""",0,null
"""Move""","""a5""","""m1""",0,null
"""Move""","""a8""","""m1""",0,null
"""Move""","""a10""","""m1""",0,null
"""Move""","""a3""","""m1""",0,null
"""Move""","""a1""","""m1""",0,null


In [7]:
events.head(10)

agent,amt,cost,event,good,port,tick
str,i64,f64,str,str,str,i64
"""a9""",1,100.05005,"""Trade""","""Wheat""","""m2""",2
"""a6""",1,100.15025,"""Trade""","""Wheat""","""m2""",2
"""a2""",1,100.250652,"""Trade""","""Wheat""","""m2""",2
"""a7""",1,100.351255,"""Trade""","""Wheat""","""m2""",2
"""a5""",1,100.452059,"""Trade""","""Wheat""","""m2""",2
"""a8""",1,100.553067,"""Trade""","""Wheat""","""m2""",2
"""a10""",1,100.654278,"""Trade""","""Wheat""","""m2""",2
"""a3""",1,100.755693,"""Trade""","""Wheat""","""m2""",2
"""a1""",1,100.857313,"""Trade""","""Wheat""","""m2""",2


In [8]:
analysis.plot_agents(agents)

alt.LayerChart(...)

In [9]:
analysis.plot_agent_locations(agents)

alt.Chart(...)

In [10]:
analysis.plot_prices_by_port(markets)

alt.LayerChart(...)

In [11]:
# analysis.plot_prices_by_port(analysis.no_agent_markets(scenario_input)) 

In [12]:
routes = analysis.make_routes(events)
routes

agent,amt,buy_cost,src,buy_tick,sell_cost,dst,sell_tick,profit,buy_price,sell_price
str,i64,f64,str,i64,f64,str,i64,f64,f64,f64
"""a10""",1,100.654278,"""m2""",2,-99.354222,"""m3""",3,-1.300056,100.654278,99.354222
"""a10""",1,99.651245,"""m3""",4,-100.351255,"""m2""",5,0.700009,99.651245,100.351255
"""a10""",1,100.553067,"""m2""",6,-99.453033,"""m3""",7,-1.100034,100.553067,99.453033
"""a10""",1,99.750648,"""m3""",8,-100.250652,"""m2""",9,0.500004,99.750648,100.250652
"""a10""",1,100.553067,"""m2""",10,-99.651245,"""m3""",11,-0.901822,100.553067,99.651245
"""a10""",1,99.651245,"""m3""",12,-100.452059,"""m2""",13,0.800814,99.651245,100.452059
"""a10""",1,100.250652,"""m2""",14,-100.05005,"""m3""",15,-0.200602,100.250652,100.05005
"""a10""",1,100.05005,"""m3""",16,-100.15025,"""m2""",17,0.1002,100.05005,100.15025
"""a10""",1,100.250652,"""m2""",18,-101.3685,"""m1""",19,1.117849,100.250652,101.3685


In [13]:
blag = routes.with_columns(pl.struct("src", "dst").alias("route"))
blag.groupby("route").agg(pl.count()).sort("count", descending=True)

route,count
struct[2],u32
"{""m3"",""m2""}",108
"{""m2"",""m3""}",91
"{""ra"",""m3""}",57
"{""m3"",""ra""}",36
"{""m2"",""m1""}",32
"{""m1"",""lb""}",19
"{""m3"",""m3""}",17
"{""m1"",""m2""}",12
"{""lb"",""la""}",11


In [14]:
analysis.plot_trades(routes)

alt.Chart(...)

In [15]:
analysis.plot_buy_and_sell_prices(routes) + analysis.plot_trades(routes)

alt.LayerChart(...)

In [16]:
curr = scenario
res = curr.markets.filter(curr.markets["tick"] == 0).select("price", "port")

utils.keyed_by(res, "port", "price")

{'m3': 100.0,
 'ra': 98.03921568627452,
 'm1': 102.04081632653062,
 'la': 102.04081632653062,
 'm2': 100.0,
 'lb': 102.04081632653062}

## Port level analysis
- How much does each port trade?
- Trade volume bucketted 
- Volume in dollars 
- Biggest trading partners (other ports)

In [17]:
## Total goods traded per port
events.groupby("port").agg(pl.sum("amt"))
# def plot_

port,amt
str,i64
"""m2""",6
"""lb""",-10
"""ra""",24
"""m1""",-11
"""la""",-8
"""m3""",5


Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)